# Projeto 1 - Ciência dos Dados

Nome: _____

Nome: _____

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\gabri\OneDrive\Área de Trabalho\Projeto_CDADOS\23-1a-cd-p1-pedro_daniel_gabriel-1\notebooks


Carregando a base de dados com os tweets classificados manualmente:

In [3]:
filename = '../data/dados.xlsx'

In [4]:
train = pd.read_excel(filename, sheet_name = 'Treinamento')
train.head(5)

,Titulo,Descrição,Pagina,Target
0,Principal província do Canadá parte para o at...,A Comissão de Valores Mobiliários de Ontário (...,77,1
1,Sheik das Criptomoedas: Justiça decreta falên...,A Justiça Estadual do Paraná decretou a falênc...,35,1
2,Protocolo assume a liderança do setor DeFi e ...,"O token LDO, da plataforma Lido Finance, sobe ...",16,0
3,99Pay libera negociação de mais quatro cripto...,"A 99Pay, carteira digital da 99, anunciou na ...",66,0
4,Casal que torrou US$ 10 milhões transferidos ...,Um casal da Austrália que ganhou por engano US...,40,2


In [5]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Titulo,Descrição,Pagina,Target
0,Governo da Argentina proíbe empresa de oferec...,"A Comissão Nacional de Valores (CNV), órgão qu...",81,2
1,Portugal planeja cobrar impostos sobre cripto...,O governo de Portugal lançou nesta segunda-fei...,41,2
2,Filho de Bolsonaro abandona metaverso e apaga...,"Renan, o filho mais novo do presidente Jair Bo...",25,2
3,Vocalista do Kiss garante que não venderá cri...,Não é uma novidade que o vocalista da banda Ki...,79,1
4,Erro faz corretora bloquear para sempre cript...,Um único erro no setor de finanças descentrali...,52,2


___
## Classificador automático


Faça aqui uma descrição do seu assunto e o que considerou como relevante ou não relevante na classificação dos tweets (Target).

Nós selecionamos 900 notícias, do "PortalBitcoin", relacionadas com o assunto "moeda". Como, em sua vasta maioria, essas notícias estão relacionadas com criptomoedas, consideramos relevante classificá-las em: favoráveis, desfavoráveis ou neutras ao mercado de criptomoedas.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

Limpeza de mensagens, removendo caracteres e substituindo-os por espaço:

In [6]:
import re

def cleanup(text):
    
    punctuation = '[!-.:?;]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

A seguir, será feita a filtragem e limpeza de cada linha, essa utilizando a função acima: 

In [37]:
good_market = []
bad_market = []
neutral_market  = []

for i in range(0,len(train)):
    t = cleanup(train.Titulo[i]).lower().split()

    if train.loc[i,'Target'] == 0:
        k = 0
        while k < len(t):
            good_market.append(t[k])
            k = k + 1
        
    elif train.loc[i,'Target'] == 1:
        l = 0
        while l < len(t):
            bad_market.append(t[l])
            l = l + 1
        
    else:
        m = 0
        while m < len(t):
            neutral_market.append(t[m])
            m = m + 1





Também realizaremos a limpeza de: aspas, | , ( , ) . Uma vez que, acreditamos que não afetará na qualidade das classificações.
Em seguida, a limpeza e classificação será refeita, porém, adicionando os novos caracteres.

In [45]:
def cleanup2(text):
    
    punctuation = '[()”“|!-.:?;]'
    pattern = re.compile(punctuation)
    text_subbed2 = re.sub(pattern, '', text)
    return text_subbed2

In [46]:
good_market_f = []
bad_market_f = []
neutral_market_f  = []

for i in range(0,len(train)):
    t = cleanup2(train.Titulo[i]).lower().split()

    if train.loc[i,'Target'] == 0:
        k = 0
        while k < len(t):
            good_market_f.append(t[k])
            k = k + 1
        
    elif train.loc[i,'Target'] == 1:
        l = 0
        while l < len(t):
            bad_market_f.append(t[l])
            l = l + 1
        
    else:
        m = 0
        while m < len(t):
            neutral_market_f.append(t[m])
            m = m + 1

Agora dividiremos as noticias em positivas, ruins, neutras e total

In [47]:
#Total
total = good_market_f + bad_market_f + neutral_market_f

#notícias e total como pd.Series
series_good = pd.Series(good_market_f)
series_bad = pd.Series(bad_market_f)
series_neutral = pd.Series(neutral_market_f)
series_total = pd.Series(total)

#Frequências absolutas
tab_good_abs = series_good.value_counts()
tab_bad_abs = series_bad.value_counts()
tab_neutral_abs = series_neutral.value_counts()
tab_total_abs = series_total.value_counts()

#Frequências relativa
tab_rel_good_abs = series_good.value_counts(True)
tab_rel_bad_abs = series_bad.value_counts(True)
tab_rel_neutral_abs = series_neutral.value_counts(True)
tab_rel_total_abs = series_total.value_counts(True)



tab_total_abs.head()

de              477
criptomoedas    408
e               150
do              126
da              112
dtype: int64

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [48]:
test_list = []


for i in range(0,len(test)):
    t = cleanup2(test.Titulo[i]).lower().split()

    test_list.append(t)


In [49]:
#Coluna de classificação pelo algoritmo

test['Target2'] = '#'

In [50]:
#P_good = len(good_market_f)/len(total) #P(good)
#P_bad = len(bad_market_f)/len(total) #P(bad)
#P_neutral = len(neutral_market_f)/len(total) #P(neutral)

#prod_frase_good = 1
#prod_frase_bad = 1
#prod_frase_neutral = 1

#for palavra in test_list:

    
    #prod_frase_good = prod_frase_good * tab_rel_good_abs [palavra]
    #prod_frase_bad = prod_frase_bad * tab_rel_bad_abs [palavra]
    #prod_frase_neutral = prod_frase_neutral * tab_rel_neutral_abs [palavra]

#probFraseDadoG = prod_frase_good
#probFraseDadoB = prod_frase_bad
#probFraseDadoN = prod_frase_neutral



Como visto no código acima, não foi possível realizar o cáclculo das probabilidades, uma vez que, podem aparecer palavras no teste que não estavam no treinamento. Sendo assim, faremos o seguinte:

In [52]:
P_good = len(good_market_f)/len(total) #P(good)
P_bad = len(bad_market_f)/len(total) #P(bad)
P_neutral = len(neutral_market_f)/len(total) #P(neutral)

probFraseDadoB = []
probFraseDadoG = []
probFraseDadoN = []
for i in range(len(test_list)):
    prod_frase_good = 1
    prod_frase_bad = 1
    prod_frase_neutral = 1


    for palavra in test_list[i]:
        if palavra not in good_market_f:
            tab_good_abs[palavra] = 0
        if palavra not in bad_market_f:
            tab_bad_abs[palavra] = 0
        if palavra not in neutral_market_f:
            tab_neutral_abs[palavra] = 0



        
        prod_frase_good = prod_frase_good * (tab_good_abs[palavra] + 1) / (len(good_market_f) +tab_total_abs.shape[0])
        prod_frase_bad = prod_frase_bad * (tab_bad_abs [palavra] + 1) / (len(bad_market_f) +tab_total_abs.shape[0])
        prod_frase_neutral = prod_frase_neutral * (tab_neutral_abs [palavra] + 1) / (len(neutral_market_f) +tab_total_abs.shape[0])

    probFraseDadoG.append(P_good * prod_frase_good)
    probFraseDadoB.append(P_bad* prod_frase_bad)
    probFraseDadoN.append(P_neutral * prod_frase_neutral)

    if probFraseDadoG[i] > probFraseDadoB[i] and probFraseDadoG[i]> probFraseDadoN[i]:
        test.loc[i,['Target2']] = 0
    elif probFraseDadoB[i] > probFraseDadoG[i] and probFraseDadoB[i]> probFraseDadoN[i]:
        test.loc[i,['Target2']] = 1
    else:
        test.loc[i,['Target2']] = 2
        
test


,Titulo,Descrição,Pagina,Target,Target2
0,Governo da Argentina proíbe empresa de oferec...,"A Comissão Nacional de Valores (CNV), órgão qu...",81,2,1
1,Portugal planeja cobrar impostos sobre cripto...,O governo de Portugal lançou nesta segunda-fei...,41,2,2
2,Filho de Bolsonaro abandona metaverso e apaga...,"Renan, o filho mais novo do presidente Jair Bo...",25,2,2
3,Vocalista do Kiss garante que não venderá cri...,Não é uma novidade que o vocalista da banda Ki...,79,1,2
4,Erro faz corretora bloquear para sempre cript...,Um único erro no setor de finanças descentrali...,52,2,1
...,...,...,...,...,...
319,“Sheik das criptomoedas” mandou R$ 4 milhões ...,"Francisley Valdevino da Silva, apelidado de “S...",41,1,1
320,Reino Unido suspende plano de monitorar carte...,"O Reino Unido, em um contraste gritante a uma ...",78,1,1
321,Regulador dos EUA diz que criptomoedas são “a...,As criptomoedas podem ser uma ameaça à seguran...,24,2,2
322,Os efeitos da ‘operação sufocamento’ dos EUA ...,A última semana foi de correção para as cripto...,6,2,2


In [53]:
Acuracia = pd.crosstab(test['Target'],test['Target2'], normalize=True, ).round(4)*100
Acuracia

Target2,0,1,2
Target,,,
0,12.65,6.79,9.57
1,4.94,19.44,13.27
2,5.56,5.86,21.91


Porcentagem de verdadeiros positivos para notícias boas para
o mercado de criptomoedas: 12,65%
Porcentagem de falsos positivos (classificadas como ruim) para notícias boas para o mercado de criptomoedas: 6,79%
Porcentagem de falsos positivos (classificadas como neutra) para notícias boas para o mercado de criptomoedas: 9,57%

Porcentagem de verdadeiros positivos para notícias ruins para
o mercado de criptomoedas: 19,44%
Porcentagem de falsos positivos (classificadas como boa) para notícias ruins para o mercado de criptomoedas: 4,94%
Porcentagem de falsos positivos (classificadas como neutra) para notícias ruins para o mercado de criptomoedas: 13,27%

Porcentagem de verdadeiros positivos para notícias neutras para o mercado de criptomoedas: 21,91%
Porcentagem de falsos positivos (classificadas como boa) para notícias neutras para o mercado de criptomoedas: 5,56%
Porcentagem de falsos positivos (classificadas como ruins) para notícias neutras para o mercado de criptomoedas: 5,86%

Acurácia: 54%

___
### Concluindo

Comparativo qualitativo:
Em todos os níveis de classificação, as notícias da planilha de teste foram classificadas, na sua maioria, corretamente, em relação a classificação manual, realizada na planilha de treinamento. Sendo assim, a acurácia obtida foi de 54%.
As notícias, antes classificadas como boa ou ruim na planilha de treinamento, que foram classificadas erroneamente na planilha de teste, majoritariamente, foram classificas como neutras.
Já, nas notícias anteriormente classificadas como neutras, as que foram classificadas de maneira incorreta, estão distribuídas, quase igualmente, entre boas e ruins.

Mensagens com dupla negação e sarcasmo:
Mensagens com essas características não possuem presença expressiva nas notícias.

Plano de expansão:
Os investidores devem continuar a investir no nosso projeto, uma vez que, nosso algoritmo possui uma acurácia significativa. Além disso, acreditamos que ainda existem melhorias que podem ser implementadas para aumentar essa variável. Desse modo, nosso programa pode ser útil para automatizar atividades de classificação.

 Não é possível alimentar a base de Treinamentos automaticamnte usando o prórpio classificador pois, nesse caso, o algoritmmo poderia classificar as frases de maneira errônea. Isso acontece pois mesmo com a boa acurácia do código, ela não é perfeita, e, assim, o erro seria levado adiante. 

 Esse classificador poderia ser usado, de maneira semehante, por uma empresa que tem o objetivo de descobrir o percentual de notícias favoráveis, desfavoráveis e neutras sobre sua marca, a partir de um banco de dados sobre comentários da empresa em uma rede socal específica. Além disso, o código poderia ser usado por um fundo de investimento que quer análisar as indicações de compra ou venda de uma determinada ação. Isso seria feito ao realizar a análise um banco de dados com indicações de diversas casas de research.

Para melhorar o código ainda mais, poderia ser relizada uma limpeza mais aprofundada na base de dados. Essa limpeza seria com as "stop-words", ou seja, palavras que, caso retiradas da base, não interferem na classificação das mesmas, por exemplo, artigos e preposições.

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas categorias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por categoria (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**